# Generación y transformación de atributos para la Competencia 01

## Carga de librerías y del dataset

In [ ]:
import duckdb
import pandas as pd
import numpy as np

In [ ]:
dataset_path = 'dataset/path/'
dataset_file = 'competencia_01_crudo.csv'

In [3]:
%%sql
create or replace table competencia_01_crudo as
select
    *
from read_csv_auto("{{dataset_path + dataset_file}}")

,Success


## Generación de la variable target

In [4]:
%%sql
create or replace table competencia_01 as
with periodos as (
    select distinct foto_mes from competencia_01_crudo
), clientes as (
    select distinct numero_de_cliente from competencia_01_crudo
), todo as (
    select numero_de_cliente, foto_mes from clientes cross join periodos
), clase_ternaria as (
    select
        c.*
        , if(c.numero_de_cliente is null, 0, 1) as mes_0
        , lead(mes_0, 1) over (partition by t.numero_de_cliente order by foto_mes) as mes_1
        , lead(mes_0, 2) over (partition by t.numero_de_cliente order by foto_mes) as mes_2
        , if(mes_1 = 0, 'baja+1', if(mes_2 = 0, 'baja+2', 'continua')) as clase_ternaria 
    from todo t
    left join competencia_01_crudo c using (numero_de_cliente, foto_mes)
) select
  * EXCLUDE (mes_0, mes_1, mes_2)
from clase_ternaria
where mes_0 = 1

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


In [5]:
%%sql
PIVOT competencia_01
on clase_ternaria
USING count(numero_de_cliente)
GROUP BY foto_mes

,foto_mes,baja+1,baja+2,continua
0,202101,622,825,160080
1,202102,831,1032,160292
2,202103,1039,951,161119
3,202104,955,1130,161333
4,202105,1134,0,162783
5,202106,0,0,164313


## Variables categóricas

Hay dos variables categóricas (Visa_status, Mater_status) que cuentan con 4 clases cada una, no ordinales ({ 0, 6, 7, 9} indica el estado de la cuenta de la tarjeta de crédito. 0 abierta, 6 en proceso de cierre, 7 en proceso avanzado de cierre, 9 cuenta cerrada.) En estos dos casos, voy a eliminar ambas variables y generar para cada una de ellas 4 variables dummies nuevas con las clases.

In [7]:
%%sql
ALTER TABLE competencia_01
ADD COLUMN visa_status_abierta INTEGER;

ALTER TABLE competencia_01
ADD COLUMN visa_status_pcierre INTEGER;

ALTER TABLE competencia_01
ADD COLUMN visa_status_pacierre INTEGER;

ALTER TABLE competencia_01
ADD COLUMN visa_status_cerrada INTEGER;

,Success


In [8]:
%%sql
UPDATE competencia_01
SET
    visa_status_abierta = CASE WHEN visa_status = 0 THEN 1 ELSE 0 END,
    visa_status_pcierre = CASE WHEN visa_status = 6 THEN 1 ELSE 0 END,
    visa_status_pacierre = CASE WHEN visa_status = 7 THEN 1 ELSE 0 END,
    visa_status_cerrada = CASE WHEN visa_status = 9 THEN 1 ELSE 0 END;

,Success


In [9]:
%%sql
ALTER TABLE competencia_01
ADD COLUMN master_status_abierta INTEGER;

ALTER TABLE competencia_01
ADD COLUMN master_status_pcierre INTEGER;

ALTER TABLE competencia_01
ADD COLUMN master_status_pacierre INTEGER;

ALTER TABLE competencia_01
ADD COLUMN master_status_cerrada INTEGER;

,Success


In [10]:
%%sql
UPDATE competencia_01
SET
    master_status_abierta = CASE WHEN master_status = 0 THEN 1 ELSE 0 END,
    master_status_pcierre = CASE WHEN master_status = 6 THEN 1 ELSE 0 END,
    master_status_pacierre = CASE WHEN master_status = 7 THEN 1 ELSE 0 END,
    master_status_cerrada = CASE WHEN master_status = 9 THEN 1 ELSE 0 END;

,Success


In [11]:
%%sql
ALTER TABLE competencia_01
DROP COLUMN visa_status;

ALTER TABLE competencia_01
DROP COLUMN master_status;

,Success


A continuación chequeamos que se crearon/eliminaron las columnas.

In [12]:
%sql SELECT * FROM competencia_01 LIMIT 1;

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_mpagominimo,clase_ternaria,visa_status_abierta,visa_status_pcierre,visa_status_pacierre,visa_status_cerrada,master_status_abierta,master_status_pcierre,master_status_pacierre,master_status_cerrada
0,609948117,202101,1,0,0,69,131,1424.59,7917.39,2019.98,...,5407.53,continua,1,0,0,0,1,0,0,0


## Atributos de tipo Entero

Hago explícito el tipo de dato porque más adelante los voy a excluir de la generación de lags y deltas.

In [ ]:
columns_to_alter = [
    "active_quarter",
    "cliente_vip",
    "tcuentas",
    "cdescubierto_preacordado",
    "tcallcenter",
    "thomebanking",
    "ccajas_transacciones",
    "tmobile_app",
    "cmobile_app_trx",
    "Master_delinquency",
    "Visa_delinquency"
]

for col in columns_to_alter:
    conn.execute(f"""
        ALTER TABLE competencia_01
        ALTER COLUMN {col} SET DATA TYPE INTEGER
    """)


## Atributos Visa + Máster

De la inspección visual del diccionario de datos se observa que los atributos asociados a las tarjetas de credito visa y master por separado se pueden agrupar en atributos generales a tener tarjeta de crédito. A continuación se van a crear atributos que los "unifiquen", sobre la base de considerar que son variables que vistas en conjunto pueden dar más información que por separado.

### Tipo fecha menor y mayor

In [ ]:
# Lista de pares de columnas (master, visa)
column_pairs = [
    ("Master_Finiciomora", "Visa_Finiciomora", "tc_finiciomora"),
    ("Master_Fvencimiento", "Visa_Fvencimiento", "tc_fvencimiento"),
    ("Master_fultimo_cierre", "Visa_fultimo_cierre", "tc_fultimocierre"),
    ("Master_fechaalta", "Visa_fechaalta", "tc_fechaalta"),
]

# Construir dinámicamente las expresiones SQL
expressions = []
for col1, col2, prefix in column_pairs:
    expressions.append(f"CAST(greatest({col1}, {col2}) AS INTEGER) AS {prefix}_mayor")
    expressions.append(f"CAST(least({col1}, {col2}) AS INTEGER) AS {prefix}_menor")

# Unir todas las expresiones en el SELECT
extra_columns_sql = ",\n    ".join(expressions)

query = f"""
CREATE OR REPLACE TABLE competencia_01_fe AS
SELECT
    *,
    {extra_columns_sql}
FROM competencia_01
"""

# Ejecutar en DuckDB
conn.execute(query)

'''# Verificar resultado
df = conn.execute("SELECT * FROM competencia_01_fe LIMIT 5").fetchdf()
print(df.head())'''


In [ ]:
# Lista de columnas a borrar
cols_to_drop = [
    "Master_Finiciomora", "Visa_Finiciomora",
    "Master_Fvencimiento", "Visa_Fvencimiento",
    "Master_fultimo_cierre", "Visa_fultimo_cierre",
    "Master_fechaalta", "Visa_fechaalta"
]

# Ejecutar un DROP COLUMN por cada columna
for col in cols_to_drop:
    conn.execute(f"ALTER TABLE competencia_01_fe DROP COLUMN {col}")

# Verificar que se borraron
#df_cols = conn.execute("PRAGMA table_info('competencia_01_fe')").fetchdf()
#print(df_cols)


### Tipo suma

In [ ]:
conn.execute("""
    CREATE OR REPLACE MACRO suma_sin_null(a, b) AS (
        ifnull(a, 0) + ifnull(b, 0)
    )
""")

In [ ]:
# Ejecutar la consulta y obtener los resultados para Visa
result = conn.execute("SELECT column_name FROM (SUMMARIZE competencia_01_fe) WHERE column_name ILIKE '%visa%'").fetchdf()
lista_column_names_visa = result['column_name'].tolist()
#print(lista_column_names_visa)

# Ejecutar la consulta y obtener los resultados para Master
result = conn.execute("SELECT column_name FROM (SUMMARIZE competencia_01_fe) WHERE column_name ILIKE '%master%'").fetchdf()
lista_column_names_master = result['column_name'].tolist()
#print(lista_column_names_master)

In [ ]:
# Generar la query de suma_sin_null dinámicamente
features_suma = []
for v_col, m_col in zip(lista_column_names_visa, lista_column_names_master):
    # Condición corregida: verificar si '_status' está en el nombre de la columna
    if '_status' not in v_col:
        # Eliminar posibles prefijos "visa", "Visa", "visa_", "Visa_"
        sufijo = v_col.replace("Visa_", "").replace("visa_", "").replace("Visa", "").replace("visa", "")
        features_suma.append(f"suma_sin_null({v_col},{m_col}) AS {sufijo}_tc")

# Construir la query completa
query = f"""
CREATE OR REPLACE TABLE competencia_01_fe AS
SELECT *,
    {', '.join(features_suma)}
FROM competencia_01_fe
"""

# Ejecutar la query
conn.execute(query)

In [ ]:
# Combinar las listas de columnas Visa y Master
columnas_a_borrar = lista_column_names_visa + lista_column_names_master

# Eliminar cada columna
for col in columnas_a_borrar:
    conn.execute(f"ALTER TABLE competencia_01_fe DROP COLUMN {col}")

## Atributos tipo ratio

In [ ]:
conn.execute("""
    CREATE OR REPLACE MACRO ratio(a, b) AS (a // (b + 0.01))
""")

In [ ]:
# Suponiendo que tienes todas las columnas de la tabla
result = conn.execute("SELECT column_name FROM (SUMMARIZE competencia_01_fe)").fetchdf()
columnas = result['column_name'].tolist()

# Separar columnas que empiezan con 'm' y 'c'
cols_m = [c for c in columnas if c.startswith('m')]
cols_c = [c for c in columnas if c.startswith('c')]

# Crear diccionario de sufijo a columna
sufijo_a_m = {c[1:]: c for c in cols_m}  # clave = sufijo sin la primera letra
sufijo_a_c = {c[1:]: c for c in cols_c}

# Generar pares donde el sufijo coincide
pares = [(sufijo_a_m[s], sufijo_a_c[s]) for s in sufijo_a_m if s in sufijo_a_c]

# Generar la query de ratios
ratios_sql = ", ".join([f"ratio({a},{b}) AS ratio_{a}_{b}" for a, b in pares])

# Crear nueva tabla con estas columnas de ratio
query = f"""
CREATE OR REPLACE TABLE competencia_01_fe AS
SELECT *,
    {ratios_sql}
FROM competencia_01_fe
"""

# Ejecutar la query
conn.execute(query)

#df_preview = conn.execute("SELECT * FROM competencia_01_fe LIMIT 5").fetchdf()
#print(df_preview.head())

## Atributos tipo delta y lag

In [ ]:
# Obtener columnas numéricas (excluyendo INTEGER y VARCHAR)
schema_df = conn.execute("DESCRIBE competencia_01_fe").fetchdf()
cols_numericas = schema_df.loc[~schema_df['column_type'].isin(['INTEGER', 'VARCHAR']), 'column_name'].tolist()

excluir = ['numero_de_cliente', 'foto_mes', 'cliente_edad', 'cliente_antiguedad']
cols_numericas = [c for c in cols_numericas if c not in excluir]

for col in cols_numericas:
    lag_col = f"{col}_lag1"
    delta_col = f"{col}_delta1"
    lag2_col = f"{col}_lag2"
    delta2_col = f"{col}_delta2"
    
    conn.execute(f"ALTER TABLE competencia_01_fe ADD COLUMN {lag_col} DOUBLE")
    conn.execute(f"ALTER TABLE competencia_01_fe ADD COLUMN {delta_col} DOUBLE")
    conn.execute(f"ALTER TABLE competencia_01_fe ADD COLUMN {lag2_col} DOUBLE")
    conn.execute(f"ALTER TABLE competencia_01_fe ADD COLUMN {delta2_col} DOUBLE")
    
    # 2️⃣ Actualizar valores con LAG y delta
    conn.execute(f"""
        UPDATE competencia_01_fe
        SET
            {lag_col} = sub.{lag_col},
            {delta_col} = sub.{delta_col},
            {lag2_col} = sub.{lag2_col},
            {delta2_col} = sub.{delta2_col}
        FROM (
            SELECT
                numero_de_cliente,
                foto_mes,
                COALESCE(LAG({col},1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), {col}) AS {lag_col},
                ({col} - COALESCE(LAG({col},1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), {col})) AS {delta_col},
                COALESCE(LAG({col},2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), {col}) AS {lag2_col},
                ({col} - COALESCE(LAG({col},2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes), {col})) AS {delta2_col}
            FROM competencia_01_fe
        ) AS sub
        WHERE competencia_01_fe.numero_de_cliente = sub.numero_de_cliente
          AND competencia_01_fe.foto_mes = sub.foto_mes
    """)

#df_preview = conn.execute("SELECT * FROM competencia_01_fe LIMIT 5").fetchdf()
#print(df_preview.head())


## Atributos tipo Mín, Máx, Promedio

In [ ]:
schema_df = conn.execute("DESCRIBE competencia_01_fe").fetchdf()

cols_numericas = schema_df.loc[
    ~schema_df['column_type'].isin(['INTEGER', 'VARCHAR']) &
    ~schema_df['column_name'].str.endswith('_lag1') &
    ~schema_df['column_name'].str.endswith('_delta1') &
    ~schema_df['column_name'].str.endswith('_lag2') &
    ~schema_df['column_name'].str.endswith('_delta2'), 
    'column_name'
].tolist()

excluir = ['numero_de_cliente', 'foto_mes', 'internet', 'cliente_edad', 'cliente_antiguedad']
cols_numericas = [c for c in cols_numericas if c not in excluir]

window_expressions = []
for col in cols_numericas:
    window_expressions.append(f"COALESCE(AVG({col}) OVER ventana_3, 0) AS {col}_avg_3")
    window_expressions.append(f"COALESCE(MAX({col}) OVER ventana_3, 0) AS {col}_max_3")
    window_expressions.append(f"COALESCE(MIN({col}) OVER ventana_3, 0) AS {col}_min_3")


query_windows = f"""
CREATE OR REPLACE TABLE competencia_01_fe AS
SELECT *,
       {', '.join(window_expressions)}
FROM competencia_01_fe
WINDOW ventana_3 AS (
    PARTITION BY numero_de_cliente
    ORDER BY foto_mes
    ROWS BETWEEN 3 PRECEDING AND CURRENT ROW
)
"""

conn.execute(query_windows)

### Variable continua_puro y baja_puro

Para el análisis exploratorio de datos inicial analicé el comportamiento de dos variables, continua_puro y baja_puro (no se usa en la notebook de la competencia).

In [ ]:
%%sql
CREATE OR REPLACE TABLE competencia_01_fe AS
WITH resumen AS (
    SELECT
        numero_de_cliente,
        BOOL_AND(clase_ternaria = 'continua') FILTER (WHERE clase_ternaria IS NOT NULL) AS solo_continua,
        BOOL_OR(clase_ternaria IN ('baja+1', 'baja+2')) AS alguna_baja,
        COUNT(*) FILTER (WHERE clase_ternaria IS NOT NULL) AS cnt_validos
    FROM competencia_01
    GROUP BY numero_de_cliente
)
SELECT
    c.*,
    CASE
        WHEN r.cnt_validos = 0 THEN NULL
        WHEN r.solo_continua THEN 'continua_puro'
        WHEN r.alguna_baja THEN 'baja_puro'
        ELSE NULL
    END AS clase_resumida
FROM competencia_01_fe c
LEFT JOIN resumen r USING (numero_de_cliente);


,Success


## Guardado en .csv

In [ ]:
dataset_export_path = dataset_path + "competencia_01_fe_1.csv"

conn.execute(f"""
COPY competencia_01_fe TO '{dataset_export_path}' (FORMAT CSV, HEADER TRUE)
""")